# Readout Mitigation



Readout errors affect quantum computation during the measurement of the qubits in a quantum device. By characterizing the readout errors, it is possible to construct a *readout error mitigator* that is used both to obtain a more accurate distribution of the outputs, and more accurate measurements of expectation value for measurables.

The readout mitigator is generated from an *assignment matrix*: a $2^n \times 2^n$ matrix $A$ such that $A_{y,x}$ is the probability to observe $y$ given the true outcome should be $x$. The assignment matrix is used to compute the *mitigation matrix* used in the readout error mitigation process itself.

A *Local readout mitigator* works under the assumption that readout errors are mostly *local*, meaning readout errors for different qubits are independent of each other. In this case, the assignment matrix is the tensor product of $n$ $2 \times 2$ matrices, one for each qubit, making it practical to store the assignment matrix in implicit form, by storing the individual $2 \times 2$ assignment matrices. The corresponding class in Qiskit is the [Local readout mitigator](https://qiskit.org/documentation/stubs/qiskit.result.LocalReadoutMitigator.html>) in ``qiskit-terra``.

 A *Correlated readout mitigator* uses the full $2^n \times 2^n$ assignment matrix, meaning it can only be used for small values of $n$. The corresponding class in Qiskit is the [Correlated readout mitigator](https://qiskit.org/documentation/stubs/qiskit.result.CorrelatedReadoutMitigator.html) in ``qiskit-terra``.

This notebook demonstrates the usage of both the local and correlated experiments to generate the corresponding mitigators.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_experiments.library import LocalReadoutError, CorrelatedReadoutError
# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

from qiskit.result.mitigation.utils import (
    expval_with_stddev,
    str2diag,
    counts_probability_vector
)

backend = AerSimulator.from_backend(FakeParis())

In [2]:
SHOTS = 1024
qubits = [0,1,2,3]
num_qubits = len(qubits)

# Standard mitigation experiment

The default mitigation experiment is *local*, meaning error probability is measured individually for each qubit. The experiment generates two circuits, one for all "0" and one for all "1" results.

In [3]:
exp = LocalReadoutError(qubits)
for c in exp.circuits():
    print(c)

         ░ ┌─┐         
   q_0: ─░─┤M├─────────
         ░ └╥┘┌─┐      
   q_1: ─░──╫─┤M├──────
         ░  ║ └╥┘┌─┐   
   q_2: ─░──╫──╫─┤M├───
         ░  ║  ║ └╥┘┌─┐
   q_3: ─░──╫──╫──╫─┤M├
         ░  ║  ║  ║ └╥┘
meas: 4/════╩══╩══╩══╩═
            0  1  2  3 
        ┌───┐ ░ ┌─┐         
   q_0: ┤ X ├─░─┤M├─────────
        ├───┤ ░ └╥┘┌─┐      
   q_1: ┤ X ├─░──╫─┤M├──────
        ├───┤ ░  ║ └╥┘┌─┐   
   q_2: ┤ X ├─░──╫──╫─┤M├───
        ├───┤ ░  ║  ║ └╥┘┌─┐
   q_3: ┤ X ├─░──╫──╫──╫─┤M├
        └───┘ ░  ║  ║  ║ └╥┘
meas: 4/═════════╩══╩══╩══╩═
                 0  1  2  3 


In [4]:
exp.analysis.set_options(plot=True)
result = exp.run(backend)
mitigator = result.analysis_results(0).value

Analysis callback <function BaseAnalysis.run.<locals>.run_analysis at 0x00000209F948FCA0> failed:
Traceback (most recent call last):
  File "C:\Users\gadia\anaconda3\envs\qiskit38\lib\site-packages\qiskit_experiments\database_service\db_experiment_data.py", line 299, in _wrapped_callback
    callback(self, **kwargs)
  File "C:\Users\gadia\anaconda3\envs\qiskit38\lib\site-packages\qiskit_experiments\framework\base_analysis.py", line 168, in run_analysis
    results, figures = analysis._run_analysis(expdata)
  File "C:\Users\gadia\anaconda3\envs\qiskit38\lib\site-packages\qiskit_experiments\library\characterization\analysis\local_readout_error_analysis.py", line 77, in _run_analysis
    figure = assignment_matrix_visualization(
  File "C:\Users\gadia\anaconda3\envs\qiskit38\lib\site-packages\qiskit_experiments\library\characterization\analysis\local_readout_error_analysis.py", line 120, in assignment_matrix_visualization
    ax.title(r"$|A - I|$")
TypeError: 'Text' object is not callable

DbExperimentEntryNotFound: 'Analysis result 0 not found. Errors: Traceback (most recent call last):\nAnalysis callback <function BaseAnalysis.run.<locals>.run_analysis at 0x00000209F948FCA0> failed: \n  File "C:\\Users\\gadia\\anaconda3\\envs\\qiskit38\\lib\\site-packages\\qiskit_experiments\\database_service\\db_experiment_data.py", line 299, in _wrapped_callback\n    callback(self, **kwargs)\nAnalysis callback <function BaseAnalysis.run.<locals>.run_analysis at 0x00000209F948FCA0> failed: \n  File "C:\\Users\\gadia\\anaconda3\\envs\\qiskit38\\lib\\site-packages\\qiskit_experiments\\framework\\base_analysis.py", line 168, in run_analysis\n    results, figures = analysis._run_analysis(expdata)\nAnalysis callback <function BaseAnalysis.run.<locals>.run_analysis at 0x00000209F948FCA0> failed: \n  File "C:\\Users\\gadia\\anaconda3\\envs\\qiskit38\\lib\\site-packages\\qiskit_experiments\\library\\characterization\\analysis\\local_readout_error_analysis.py", line 77, in _run_analysis\n    f

The resulting measurement matrix can be illustrated by comparing it to the identity.

In [ ]:
result.figure(0)

## Mitigation matrices

The individual mitigation matrices can be read off the mitigator.

In [ ]:
for m in mitigator._mitigation_mats:
    print(m)
    print()

## Mitigation Example

In [ ]:
qc = QuantumCircuit(num_qubits)
qc.h(0)
for i in range(1, num_qubits):
    qc.cx(i - 1, i)
qc.measure_all()

In [ ]:
counts = backend.run(qc, shots=SHOTS, seed_simulator=42, method="density_matrix").result().get_counts()
unmitigated_probs = {label: count / SHOTS for label, count in counts.items()}

In [ ]:
mitigated_quasi_probs = mitigator.quasi_probabilities(counts)
mitigated_stddev = mitigated_quasi_probs._stddev_upper_bound
mitigated_probs = (mitigated_quasi_probs.nearest_probability_distribution().binary_probabilities())

### Probabilities

In [ ]:
legend = ['Mitigated Probabilities', 'Unmitigated Probabilities']
plot_histogram([mitigated_probs, unmitigated_probs], legend=legend, sort="value_desc", bar_labels=False)

## Expectation value

In [ ]:
diagonal_labels = ["ZZZZ", "ZIZI", "IZII", "1ZZ0"]
ideal_expectation = []
diagonals = [str2diag(d) for d in diagonal_labels]
qubit_index = {i: i for i in range(num_qubits)}
unmitigated_probs_vector, _ = counts_probability_vector(unmitigated_probs, qubit_index=qubit_index)
unmitigated_expectation = [expval_with_stddev(d, unmitigated_probs_vector, SHOTS) for d in diagonals]
mitigated_expectation = [mitigator.expectation_value(counts, d) for d in diagonals]

In [ ]:
mitigated_expectation_values, mitigated_stddev = zip(*mitigated_expectation)
unmitigated_expectation_values, unmitigated_stddev = zip(*unmitigated_expectation)
legend = ['Mitigated Expectation', 'Unmitigated Expectation']
fig, ax = plt.subplots()
X = np.arange(4)
ax.bar(X + 0.00, mitigated_expectation_values, yerr=mitigated_stddev, color='b', width = 0.25, label="Mitigated Expectation")
ax.bar(X + 0.25, unmitigated_expectation_values, yerr=unmitigated_stddev, color='g', width = 0.25, label="Unmitigated Expectation")
ax.set_xticks([0.125 + i for i in range(len(diagonals))])
ax.set_xticklabels(diagonal_labels)
ax.legend()

## Correlated readout mitigation

In correlated readout mitigation on $n$ qubits, a circuit is generated for each of the possible $2^n$ combinations of "0" and "1". This results in more accurate mitigation in the case where the readout errors are correlated and not independent, but requires a large amount of circuits and storage space, and so is infeasible for more than a few qubits.

In [ ]:
qubits = [0,3]
num_qubits = len(qubits)
exp = CorrelatedReadoutError(qubits)
for c in exp.circuits():
    print(c)

In [ ]:
import qiskit.tools.jupyter
%qiskit_copyright